In [1]:
from gensim.models import KeyedVectors
import pandas as pd
import pymorphy3
from scipy.stats import spearmanr
import numpy as np

morph = pymorphy3.MorphAnalyzer()

In [2]:
cbow = KeyedVectors.load_word2vec_format(
    "../models/ruscorpora_upos_cbow_300_20_2019/model.bin",
    binary=True,
    unicode_errors="ignore",
)
skipgram = KeyedVectors.load_word2vec_format(
    "../models/tayga_upos_skipgram_300_2_2019/model.bin",
    binary=True,
    unicode_errors="ignore",
)

In [3]:
vector = cbow["москва_NOUN"]

In [4]:
df = pd.read_csv("hj-wordsim353-similarity.csv")
df

,word1,word2,sim
0,маг,волшебник,0.958333
1,машина,автомобиль,0.952381
2,мальчик,парень,0.952381
3,доллар,бакс,0.952381
4,расчет,вычисление,0.916667
...,...,...,...
197,фонд,яйцо,0.000000
198,арафат,джексон,0.000000
199,петушок,путешествие,0.000000
200,полдень,веревка,0.000000


In [5]:
def get_cosine_similarity(model, word1, word2):
    pos_mapping = {
        "NOUN": "NOUN",  # имя существительное
        "ADJF": "ADJ",  # имя прилагательное (полное)
        "ADJS": "ADJ",  # имя прилагательное (краткое)
        "COMP": "ADJ",  # компаратив
        "VERB": "VERB",  # глагол (личная форма)
        "INFN": "VERB",  # глагол (инфинитив)
        "PRTF": "ADJ",  # причастие (полное)
        "PRTS": "ADJ",  # причастие (краткое)
        "GRND": "VERB",  # деепричастие
        "NUMR": "NUM",  # числительное
        "ADVB": "ADV",  # наречие
        "NPRO": "PRON",  # местоимение-существительное
        "PRED": "ADV",  # предикатив
        "PREP": "ADP",  # предлог
        "CONJ": "CCONJ",  # союз
        "PRCL": "PART",  # частица
        "INTJ": "INTJ",  # междометие
    }
    try:
        parsed1 = morph.parse(word1)[0]
        lemma1 = parsed1.normal_form
        w1 = f"{lemma1}_{pos_mapping[parsed1.tag.POS]}"
        parsed2 = morph.parse(word2)[0]
        lemma2 = parsed2.normal_form
        w2 = f"{lemma2}_{pos_mapping[parsed2.tag.POS]}"
        return model.similarity(w1, w2)
    except KeyError:
        return np.nan

In [6]:
def test_model(df, model):
    cosine_similarities = []
    for i, row in df.iterrows():
        word1 = row['word1']
        word2 = row['word2']
        
        similarity = get_cosine_similarity(model, word1, word2)
        cosine_similarities.append(similarity)

    df['cosine_similarity'] = cosine_similarities
    df_clean = df.dropna(subset=['cosine_similarity'])

    if len(df_clean) > 0:
        spearman_corr, p_value = spearmanr(df_clean['sim'], df_clean['cosine_similarity'])
        
        print("\n" + "="*50)
        print("РЕЗУЛЬТАТЫ:")
        print(f"Всего пар: {len(df)}")
        print(f"Успешно обработано: {len(df_clean)}")
        print(f"Корреляция Спирмена: {spearman_corr:.4f}")
        print(f"p-value: {p_value:.4f}")
        print("="*50)
    return cosine_similarities

In [7]:
cbow_sim = test_model(df, cbow)



РЕЗУЛЬТАТЫ:
Всего пар: 202
Успешно обработано: 187
Корреляция Спирмена: 0.7234
p-value: 0.0000


In [8]:
skipgram_sim = test_model(df, skipgram)



РЕЗУЛЬТАТЫ:
Всего пар: 202
Успешно обработано: 190
Корреляция Спирмена: 0.7862
p-value: 0.0000


In [9]:
clean_cbowsim, clean_skipgram_sim = [], []

for i in range(len(cbow_sim)):
    if not np.isnan(cbow_sim[i]) and not np.isnan(skipgram_sim[i]):
        clean_cbowsim.append(cbow_sim[i])
        clean_skipgram_sim.append(skipgram_sim[i])

spearman_corr, p_value = spearmanr(clean_cbowsim, clean_skipgram_sim)
        
print("\n" + "="*50)
print("РЕЗУЛЬТАТЫ:")
print(f"Всего пар: {len(cbow_sim)}")
print(f"Успешно обработано: {len(clean_cbowsim)}")
print(f"Корреляция Спирмена: {spearman_corr:.4f}")
print(f"p-value: {p_value:.4f}")
print("="*50)


РЕЗУЛЬТАТЫ:
Всего пар: 202
Успешно обработано: 187
Корреляция Спирмена: 0.9216
p-value: 0.0000


# Отчет по сравнительному анализу моделей Word2Vec

В рамках лабораторной работы были загружены две модели эмбеддингов с 
сайта rusvectores.org:
- **ruscorpora_upos_cbow_300_20_2019** (CBOW архитектура)
- **tayga_upos_skipgram_300_2_2019** (Skip-gram архитектура)

Для оценки качества моделей использовался датасет `hj-wordsim353-similarity`,
 содержащий пары слов с человеческими оценками семантической близости.

## Результаты оценки

### Корреляция с человеческими оценками (метрика Спирмена):
- **CBOW модель**: 0.7234
- **Skip-gram модель**: 0.7862

### Взаимная корреляция моделей:
- **Корреляция между предсказаниями моделей**: 0.9216

## Анализ результатов

1. **Качество моделей**: Модель `tayga_upos_skipgram_300_2_2019` продемонстрировала 
более высокое качество, показав корреляцию 0.7862 с человеческими оценками 
против 0.7234 у CBOW модели.

2. **Схожесть моделей**: Высокая взаимная корреляция (0.9216) свидетельствует о том, 
что обе модели в целом схожим образом оценивают семантическую близость слов.

3. **Размер словаря**: Модель Skip-gram в данном датасете имеет несколько больший 
словарь (носитель) - 190 пар слов против пар 187 у CBOW модели.

## Вывод

На основе проведенного анализа можно заключить, что модель 
**tayga_upos_skipgram_300_2_2019** показывает несколько лучшие результаты в задаче 
определения семантической близости слов, что согласуется с теоретическими ожиданиями, 
поскольку архитектура Skip-gram обычно лучше справляется с редкими словами и сложными 
семантическими отношениями.